In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('z_train.csv')
df_test = pd.read_csv('z_test.csv')

In [3]:
df_train_copy = df_train.copy()
df_test_copy = df_test.copy()

In [4]:
df_train_copy[df_train_copy.target==1].shape

(3824, 14)

In [5]:
df_train_copy[df_train_copy.target==0].shape

(11502, 14)

In [6]:
df_test_copy.drop(['enrollee_id', 'city','company_size', 'enrolled_university', 'major_discipline'], axis=1 , inplace = True)

In [7]:
df_train_copy.drop(['enrollee_id', 'city','company_size', 'enrolled_university', 'major_discipline'], axis=1 , inplace = True)


In [8]:
df_train_copy.dropna(inplace=True)

In [9]:
df_train_copy.shape

(8081, 9)

In [10]:
df_train.shape

(15326, 14)

In [11]:
def cambiar_mayor(num):
    num= str(num)
    if '>' in num:
        num = num.replace('>','')
        return num
    
    if '<' in num:
        num = num.replace('<1','0')
        return num
    
    if 'nan' in num:
        num = num.replace('nan','')
        return num
    
    else:
        return num

In [12]:
df_train_copy.experience = df_train_copy.experience.apply(cambiar_mayor)

In [13]:
df_train_copy['experience'] = pd.to_numeric(df_train_copy.experience)

In [14]:
df_test_copy.experience = df_train_copy.experience.apply(cambiar_mayor)
df_test_copy['experience'] = pd.to_numeric(df_train_copy.experience)

In [15]:
def cambiar_num(num):
    num= str(num)
    if '>4' in num:
        num = num.replace('>4','5')
        return num
    
    if 'never' in num:
        num = num.replace('never','0')
        return num
    
    if 'nan' in num:
        num = num.replace('nan','')
        return num
    
    else:
        return num

In [16]:
df_train_copy.last_new_job = df_train_copy.last_new_job.apply(cambiar_num)

In [17]:
df_train_copy['last_new_job'] = pd.to_numeric(df_train_copy.last_new_job)

In [18]:
df_test_copy.last_new_job = df_train_copy.last_new_job.apply(cambiar_num)
df_test_copy['last_new_job'] = pd.to_numeric(df_train_copy.last_new_job)

In [19]:
df_train_copy.columns

Index(['city_development_index', 'gender', 'relevent_experience',
       'education_level', 'experience', 'company_type', 'last_new_job',
       'training_hours', 'target'],
      dtype='object')

In [20]:
df_train_copy.experience.value_counts()

20    1680
5      542
6      504
10     490
9      464
4      461
7      443
3      426
15     357
8      350
11     340
2      315
14     305
16     268
12     235
13     197
17     178
19     157
18     142
1      131
0       96
Name: experience, dtype: int64

In [21]:
pip install feature_engine

Note: you may need to restart the kernel to use updated packages.


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from feature_engine import imputation as imp
from feature_engine import encoding as enc
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.encoding import OneHotEncoder
from feature_engine import imputation as mdi


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from feature_engine.wrappers import SklearnTransformerWrapper


from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import sklearn.metrics

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df_train_copy.drop(columns = ['target']), df_train_copy.target, test_size = 0.2, random_state = 830)

In [24]:
df_train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8081 entries, 1 to 15324
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  8081 non-null   float64
 1   gender                  8081 non-null   object 
 2   relevent_experience     8081 non-null   object 
 3   education_level         8081 non-null   object 
 4   experience              8081 non-null   int64  
 5   company_type            8081 non-null   object 
 6   last_new_job            8081 non-null   int64  
 7   training_hours          8081 non-null   int64  
 8   target                  8081 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 889.4+ KB


In [25]:
df_train_copy.training_hours

1         47
5         24
6         18
7         46
10       108
        ... 
15317     36
15318     23
15320     25
15323     44
15324     97
Name: training_hours, Length: 8081, dtype: int64

In [26]:
pipeline_gilda = Pipeline([('mean_imputation', mdi.MeanMedianImputer(imputation_method = 'median',
                                                                             variables = ['experience', 'last_new_job','training_hours'])),
                                   ('mode_imputation', mdi.CategoricalImputer(imputation_method = 'missing',
                                    variables = ['education_level', 'relevent_experience','gender','company_type'])),
                                   ('escalar', SklearnTransformerWrapper(StandardScaler(), variables=['experience', 'last_new_job','training_hours'])),
                                   ('one_hot', enc.OneHotEncoder(variables = ['education_level', 'relevent_experience','gender','company_type'])),                         
                                   ], verbose = True)
pipeline_gilda.fit(X_train, y_train)

[Pipeline] ... (step 1 of 4) Processing mean_imputation, total=   0.0s
[Pipeline] ... (step 2 of 4) Processing mode_imputation, total=   0.0s
[Pipeline] ........... (step 3 of 4) Processing escalar, total=   0.0s
[Pipeline] ........... (step 4 of 4) Processing one_hot, total=   0.0s


Pipeline(steps=[('mean_imputation',
                 MeanMedianImputer(variables=['experience', 'last_new_job',
                                              'training_hours'])),
                ('mode_imputation',
                 CategoricalImputer(variables=['education_level',
                                               'relevent_experience', 'gender',
                                               'company_type'])),
                ('escalar',
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['experience',
                                                      'last_new_job',
                                                      'training_hours'])),
                ('one_hot',
                 OneHotEncoder(variables=['education_level',
                                          'relevent_experience', 'gender',
                                          'company_type']))],
         verbose=True)

In [27]:
test_1 = pipeline_gilda.transform(df_test_copy)

In [28]:
test_1.shape

(3832, 20)

In [29]:
data_1 = pipeline_gilda.transform(X_train)

In [30]:
data_1.shape

(6464, 20)

In [31]:
X_train.head(5)

,city_development_index,gender,relevent_experience,education_level,experience,company_type,last_new_job,training_hours
6206,0.896,Male,Has relevent experience,Graduate,7,Pvt Ltd,5,34
1931,0.899,Male,Has relevent experience,Graduate,3,Pvt Ltd,1,155
1023,0.624,Male,Has relevent experience,Graduate,4,Pvt Ltd,1,50
2843,0.884,Other,Has relevent experience,Masters,10,Pvt Ltd,5,141
10360,0.939,Male,Has relevent experience,Graduate,8,Pvt Ltd,1,31


In [32]:
data_1.head(5)

,city_development_index,experience,last_new_job,training_hours,education_level_Graduate,education_level_Masters,education_level_Primary School,education_level_High School,education_level_Phd,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,gender_Male,gender_Other,gender_Female,company_type_Pvt Ltd,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Public Sector,company_type_Other
6206,0.896,-0.650472,1.625735,-0.521249,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0
1931,0.899,-1.287492,-0.760990,1.482543,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0
1023,0.624,-1.128237,-0.760990,-0.256284,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0
2843,0.884,-0.172707,1.625735,1.250699,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0
10360,0.939,-0.491217,-0.760990,-0.570929,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0


In [33]:
data_1.columns

Index(['city_development_index', 'experience', 'last_new_job',
       'training_hours', 'education_level_Graduate', 'education_level_Masters',
       'education_level_Primary School', 'education_level_High School',
       'education_level_Phd', 'relevent_experience_Has relevent experience',
       'relevent_experience_No relevent experience', 'gender_Male',
       'gender_Other', 'gender_Female', 'company_type_Pvt Ltd',
       'company_type_Early Stage Startup', 'company_type_Funded Startup',
       'company_type_NGO', 'company_type_Public Sector', 'company_type_Other'],
      dtype='object')

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
grid_forestrest = RandomForestClassifier()
params = {'max_depth': [8, 6, 5, 4], 'n_estimators':[47, 46, 45, 44]}
grid_forest = GridSearchCV(grid_forestrest, params).fit(data_1, y_train)

In [45]:
grid_forest.best_estimator_

RandomForestClassifier(max_depth=8, n_estimators=45)

In [46]:
best_forest = RandomForestClassifier(max_depth=8, n_estimators=45).fit(data_1, y_train)

In [47]:
pd.DataFrame([best_forest.feature_importances_], columns = best_forest.feature_names_in_).T

,0
city_development_index,0.639767
experience,0.103422
last_new_job,0.044554
training_hours,0.084414
education_level_Graduate,0.012793
education_level_Masters,0.008382
education_level_Primary School,0.002153
education_level_High School,0.008761
education_level_Phd,0.003918
relevent_experience_Has relevent experience,0.011197


In [48]:
pred_y = best_forest.predict(data_1)

In [49]:
pred_y

array([0., 0., 1., ..., 0., 0., 0.])

In [50]:
metrics.roc_auc_score(y_train, pred_y)

0.7047219664222291

In [51]:
from sklearn.metrics import roc_auc_score

In [52]:
metrics.accuracy_score(y_train, pred_y)

0.8819616336633663

In [53]:
test_2 = pipeline_gilda.transform(X_test)

In [54]:
pred_y_test = best_forest.predict(test_2)

In [55]:
metrics.roc_auc_score(y_test, pred_y_test)

0.6692584020444573

In [56]:
df_test_copy = df_test.copy(deep=False)

In [57]:
prediccion = df_test_copy.copy(deep=False)

In [58]:
prediccion.drop(['city', 'city_development_index', 'gender',
       'relevent_experience', 'education_level', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours'],axis=1, inplace=True)

In [59]:
pred_y_test = best_forest.predict_proba(test_1)

In [60]:
 best_forest.predict(test_1)

array([0., 0., 0., ..., 0., 0., 0.])

In [61]:
pred_y_test[:-1]

array([[0.8611586 , 0.1388414 ],
       [0.91844615, 0.08155385],
       [0.82736673, 0.17263327],
       ...,
       [0.91032447, 0.08967553],
       [0.90941085, 0.09058915],
       [0.87835145, 0.12164855]])

In [62]:
datos=[]
for x in pred_y_test:
    datos.append(x[1])

In [63]:
target = pd.DataFrame(datos)

In [64]:
prediccion['target'] = target

In [65]:
prediccion

,enrollee_id,enrolled_university,major_discipline,target
0,23603,Full time course,STEM,0.138841
1,22499,Full time course,NaN,0.081554
2,10465,no_enrollment,STEM,0.172633
3,8293,no_enrollment,STEM,0.110651
4,4246,no_enrollment,STEM,0.107423
...,...,...,...,...
3827,8880,Part time course,Humanities,0.137411
3828,7886,no_enrollment,STEM,0.089676
3829,12279,no_enrollment,STEM,0.090589
3830,5326,no_enrollment,NaN,0.121649


In [66]:
prediccion.drop(['enrolled_university', 'major_discipline'],axis=1, inplace=True)

In [67]:
prediccion

,enrollee_id,target
0,23603,0.138841
1,22499,0.081554
2,10465,0.172633
3,8293,0.110651
4,4246,0.107423
...,...,...
3827,8880,0.137411
3828,7886,0.089676
3829,12279,0.090589
3830,5326,0.121649


In [68]:
prediccion.to_csv('prediccion4_gilda.csv', index=False)